In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install transformers datasets scikit-learn torch

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer
from transformers.training_args import TrainingArguments
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import EarlyStoppingCallback


In [ ]:
train_path = "/content/drive/MyDrive/PMML/dev_phase/subtask1/train/eng.csv"
dev_path   = "/content/drive/MyDrive/PMML/dev_phase/subtask1/dev/eng.csv"

In [ ]:
df = pd.read_csv(train_path)
df_dev = pd.read_csv(dev_path)

In [ ]:
df.head()

,id,text,polarization
0,eng_973938b90b0ff5d87d35a582f83f5c89,is defending imperialism in the dnd chat,0
1,eng_07dfd4600426caca6e2c5883fcbea9ea,Still playing with this. I am now following Ra...,0
2,eng_f14519ff2302b6cd47712073f13bc461,.senate.gov Theres 3 groups out there Republic...,0
3,eng_e48b7e7542faafa544ac57b64bc80daf,"""ABC MD, David Anderson, said the additional f...",0
4,eng_7c581fb77bce8033aeba3d6dbd6273eb,"""bad people"" I have some conservative values s...",0


In [ ]:
df_dev.head()

,id,text,polarization
0,eng_f66ca14d60851371f9720aaf4ccd9b58,God is with Ukraine and Zelensky,NaN
1,eng_3a489aa7fed9726aa8d3d4fe74c57efb,"4 Dems, 2 Republicans Luzerne County Council s...",NaN
2,eng_95770ff547ea5e48b0be00f385986483,Abuse Survivor Recounts Her Struggles at YWCA ...,NaN
3,eng_2048ae6f9aa261c48e6d777bcc5b38bf,"After Rwanda, another deportation camp disaster",NaN
4,eng_07781aa88e61e7c0a996abd1e5ea3a20,Another plea in Trump election interference probe,NaN


In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["polarization"])

dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "test": Dataset.from_pandas(test_df)
})

In [ ]:
train_df

,id,text,polarization
2416,eng_953175fbdf11ba464912586a5b9500da,Epic treason by all Republicans.,1
1128,eng_8ec42880054b58ddb3f0a9d288496c2c,This map is an example by using our legend on ...,0
1198,eng_7273175cf6d717426d65a8b217384830,US President Biden Authorizes 571 Million In M...,0
2644,eng_a8d8e4963eee903979197b4b80b56eb0,"abolish your he second amendment, fuck this shit",1
584,eng_64fa6b7a7ac9854d09b77dacf7d3839f,In aggregate this is true but I dont want to r...,0
...,...,...,...
774,eng_6684167e3356f0eb6c2e17e07478ecee,Mike Pence? I couldnt care any less.,0
1252,eng_6e214ea8e214af8cfb2f331b7be5a8bd,What issues are most important to likely GOP,0
2038,eng_696a0e65c800e613b02e0f92f34d2a70,Or conversely repress red states votes...,1
1031,eng_9b3e962b13b6424c6d33a0d5776c0bad,Stories from Refugees Who Found Their Home,0


In [ ]:
test_df

,id,text,polarization
1872,eng_3fad6ff3bc1e061cc1f79c1898e6dd32,I am boycotting MSNBC until they stop talking ...,1
2261,eng_8e5d02c9af466240fb8b41b670dfc5d2,"What even is ""progressive populism""? Socialism...",1
1194,eng_9bdab77421afa033967336f3154b8104,Urgent Loud explosions in the West Bank due to...,0
2466,eng_39662e0c86421de7d12464e5aea363d5,If Mary traveled through the west bank shed ru...,1
2549,eng_3d2b4cc1bd747eac22aad9762e6e4fb2,Seeing democrats melt down over Ted Cruz has b...,1
...,...,...,...
343,eng_f610de1f2cea9e85c0e10f62f792a74b,Election Day as national holiday. Automatic vo...,0
725,eng_ac626ee4768055bbcdf6606ba0ebe5ed,Latino Texas Democrats in U.S. House push back...,0
391,eng_a77f98e693a787654a4943c1e6ffe400,"Football: #2 Warriors Blank #3 Gophers, Retake...",0
207,eng_721a0d19807b31eab874abace9919de2,Colorado voters sue to disqualify Donald Trump,0


# Fine-Tune model xlm-roberta

In [ ]:
model_name = "xlm-roberta-base"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

def preprocess(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

encoded_dataset = dataset.map(preprocess, batched=True)
encoded_dataset = encoded_dataset.remove_columns(["text"])
encoded_dataset = encoded_dataset.rename_column("polarization", "labels")
encoded_dataset.set_format("torch")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/2140 [00:00<?, ? examples/s]

Map:   0%|          | 0/536 [00:00<?, ? examples/s]

In [ ]:
num_labels = len(df["polarization"].unique())

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels
)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), dim=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "precision": precision_score(labels, preds, average="macro"),
        "recall": recall_score(labels, preds, average="macro"),
        "f1": f1_score(labels, preds, average="macro")
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_xlm-roberta-base",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_steps=500,
    report_to="none",               # interval save
    save_total_limit=2,               # maksimal checkpoint
    logging_steps=50,
    load_best_model_at_end=True,      # WAJIB untuk early stopping
    metric_for_best_model="f1",       # sesuaikan dg compute_metrics
    greater_is_better=True            # F1 lebih besar = lebih baik
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[
        EarlyStoppingCallback(early_stopping_patience=2)
    ]
)

/tmp/ipython-input-2098548275.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.581100,0.542941,0.763060,0.747329,0.744776,0.745976
2,0.491100,0.528335,0.777985,0.765994,0.778607,0.769472
3,0.358900,0.541347,0.796642,0.784036,0.778607,0.781058
4,0.287400,0.549319,0.777985,0.764317,0.773632,0.767659


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.581100,0.542941,0.763060,0.747329,0.744776,0.745976
2,0.491100,0.528335,0.777985,0.765994,0.778607,0.769472
3,0.358900,0.541347,0.796642,0.784036,0.778607,0.781058
4,0.287400,0.549319,0.777985,0.764317,0.773632,0.767659
5,0.222200,0.774607,0.792910,0.779167,0.778607,0.778884


TrainOutput(global_step=670, training_loss=0.40309667017922474, metrics={'train_runtime': 546.8244, 'train_samples_per_second': 39.135, 'train_steps_per_second': 2.451, 'total_flos': 703822073088000.0, 'train_loss': 0.40309667017922474, 'epoch': 5.0})

In [ ]:
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.5413468480110168, 'eval_accuracy': 0.7966417910447762, 'eval_precision': 0.7840358527131783, 'eval_recall': 0.7786069651741294, 'eval_f1': 0.7810580594872718, 'eval_runtime': 3.6471, 'eval_samples_per_second': 146.966, 'eval_steps_per_second': 9.322, 'epoch': 5.0}


In [ ]:
trainer.save_model("./saved_model_xlm-roberta_f1_81")
tokenizer.save_pretrained("./saved_model_xlm-roberta_f1_81")

('./saved_model_xlm-roberta_f1_81/tokenizer_config.json',
 './saved_model_xlm-roberta_f1_81/special_tokens_map.json',
 './saved_model_xlm-roberta_f1_81/sentencepiece.bpe.model',
 './saved_model_xlm-roberta_f1_81/added_tokens.json',
 './saved_model_xlm-roberta_f1_81/tokenizer.json')

## Evaluasi

In [ ]:
import numpy as np

predictions = trainer.predict(encoded_dataset["test"])
logits = predictions.predictions
labels = predictions.label_ids
preds = np.argmax(logits, axis=-1)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(labels, preds))

              precision    recall  f1-score   support

           0       0.83      0.85      0.84       335
           1       0.74      0.71      0.72       201

    accuracy                           0.80       536
   macro avg       0.78      0.78      0.78       536
weighted avg       0.80      0.80      0.80       536



## testing data dev

In [ ]:
dataset_dev = DatasetDict({
    "dev": Dataset.from_pandas(df_dev)
})

encoded_dataset_dev = dataset_dev.map(preprocess, batched=True)
encoded_dataset_dev = encoded_dataset_dev.remove_columns(["text"])  # keep only input_ids etc.

Map:   0%|          | 0/133 [00:00<?, ? examples/s]

In [ ]:
predictions = trainer.predict(encoded_dataset_dev["dev"])
logits = predictions.predictions
labels = predictions.label_ids
preds = np.argmax(logits, axis=-1)

In [ ]:
df_dev["polarization"] = preds
df_dev.head()

,id,text,polarization
0,eng_f66ca14d60851371f9720aaf4ccd9b58,God is with Ukraine and Zelensky,0
1,eng_3a489aa7fed9726aa8d3d4fe74c57efb,"4 Dems, 2 Republicans Luzerne County Council s...",0
2,eng_95770ff547ea5e48b0be00f385986483,Abuse Survivor Recounts Her Struggles at YWCA ...,0
3,eng_2048ae6f9aa261c48e6d777bcc5b38bf,"After Rwanda, another deportation camp disaster",0
4,eng_07781aa88e61e7c0a996abd1e5ea3a20,Another plea in Trump election interference probe,0


In [ ]:
df_dev.to_csv("/content/drive/MyDrive/PMML/dev_phase/subtask1/dev_xlm-roberta.csv")